In [1]:
import tensorflow as tf
from tensorflow_serving.apis import predict_pb2
from tensorflow_serving.apis import prediction_service_pb2
from keras.models import load_model
from keras.models import model_from_json

Using TensorFlow backend.


In [2]:
sess = tf.InteractiveSession()

In [3]:
# model = load_model('hr_keras.h5')
json_file = open('model_yolo.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model_yolo.h5")
print("Loaded model from disk")

Loaded model from disk


In [5]:
print(loaded_model.output)

Tensor("lambda_1/Identity:0", shape=(?, 9, 9, 5, 8), dtype=float32)


In [12]:
# classification_signature = (
#     tf.saved_model.signature_def_utils.build_signature_def(
#         inputs={
#             tf.saved_model.signature_constants.CLASSIFY_INPUTS:
#               model.input
#         },
#         outputs={
#             tf.saved_model.signature_constants.CLASSIFY_OUTPUT_CLASSES:
#               classification_outputs_classes,
#             tf.saved_model.signature_constants.CLASSIFY_OUTPUT_SCORES:
#               classification_outputs_scores
#         },
#         method_name=tf.saved_model.signature_constants.CLASSIFY_METHOD_NAME))

In [6]:
# model_input = tf.saved_model.utils.build_tensor_info(model.input)
# model_output= tf.saved_model.utils.build_tensor_info(model.output)
model_input = tf.saved_model.utils.build_tensor_info(loaded_model.input[0])
model_output= tf.saved_model.utils.build_tensor_info(loaded_model.output)

In [7]:
prediction_signature = (
      tf.saved_model.signature_def_utils.build_signature_def(
          inputs={'images': model_input},
          outputs={'bounding_box': model_output},
          method_name=tf.saved_model.signature_constants.PREDICT_METHOD_NAME))

In [8]:
builder = tf.saved_model.builder.SavedModelBuilder('./models_yolo/yolo/1')

builder.add_meta_graph_and_variables(
    sess, [tf.saved_model.tag_constants.SERVING],
    signature_def_map={
    tf.saved_model.signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY:
        prediction_signature
    })

# Save the model so we can serve it with a model server :)
builder.save()

INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: ./models_yolo/yolo/1/saved_model.pb


b'./models_yolo/yolo/1/saved_model.pb'

In [9]:
print(loaded_model.input)

[<tf.Tensor 'input_1:0' shape=(?, 300, 300, 3) dtype=float32>, <tf.Tensor 'input_2:0' shape=(?, 1, 1, 1, 2, 4) dtype=float32>]


In [14]:
print(model_input)

name: "input_1:0"
dtype: DT_FLOAT
tensor_shape {
  dim {
    size: -1
  }
  dim {
    size: -1
  }
  dim {
    size: -1
  }
  dim {
    size: 3
  }
}

